# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
location = cities_df[["Lat", "Lng"]]
location

# Store Humidity in humidity
humidity = cities_df["Humidity"]
humidity

0      72
1      74
2      73
3      39
4      76
       ..
543    32
544     4
545    62
546    93
547    64
Name: Humidity, Length: 548, dtype: int64

In [17]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down the cities to fit weather conditions
narrow_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()

narrow_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# Create a hotel_df
hotel_df = narrow_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    city = row['City']
    latitude = row['Lat']
    longitude = row['Lng']

    # add keyword to params dict
    params['location'] = (f"{latitude},{longitude}")

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} Hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel"], inplace=True)
hotel_df

Retrieving Results for Index 70: ponta do sol.
Closest ponta do sol Hotel is Escarpas Resort.
------------
Retrieving Results for Index 88: cidreira.
Closest cidreira Hotel is Hotel Castelo.
------------
Retrieving Results for Index 138: morondava.
Closest morondava Hotel is Palissandre Cote Ouest resort & SPA.
------------
Retrieving Results for Index 139: vaini.
Missing field/result... skipping.
------------
Retrieving Results for Index 176: nador.
Closest nador Hotel is Hôtel Mercure Rif Nador.
------------
Retrieving Results for Index 297: mogok.
Closest mogok Hotel is MOGOK HILL HOTEL.
------------
Retrieving Results for Index 444: birjand.
Closest birjand Hotel is هتل پرويز.
------------


,City,Country,Lat,Lng,Hotel Name,Hotel
70,ponta do sol,BR,-20.63,-46.00,Escarpas Resort,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,NaN,Hotel Castelo
138,morondava,MG,-20.30,44.28,NaN,Palissandre Cote Ouest resort & SPA
176,nador,MA,35.17,-2.93,NaN,Hôtel Mercure Rif Nador
297,mogok,MM,22.92,96.51,NaN,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,NaN,هتل پرويز


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
location = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(location, info_box_content = hotel_info)


# Display figure
fig.add_layer(markers)

In [26]:
# Display the map
fig

Figure(layout=FigureLayout(height='420px'))